In [8]:
from behavior_utils import *
import matplotlib.pyplot as plt

### Initialize paths


In [10]:
# Get paths to relevant files
ttl_paths = get_file_paths(r'S:/clickbait-visual/preprocessed', 'npy', 'ttl', session_type="v", print_paths=True, print_n=np.inf)
video_paths = get_file_paths(r'S:/clickbait-visual/tetrode_ob-hc', 'avi', '', session_type="v", print_paths=False, print_n=3)
timestamps = get_file_paths(r'S:/clickbait-visual/tetrode_ob-hc', 'csv', 'timestamp', session_type="v", print_paths=False, print_n=3)
event_paths_a = get_file_paths(r'S:/clickbait-visual/tetrode_ob-hc', 'csv', '_eventsA', session_type="v", print_paths=False, print_n=3)
event_paths_b = get_file_paths(r'S:/clickbait-visual/tetrode_ob-hc', 'csv', '_eventsB', session_type="v", print_paths=False, print_n=3)

# Root directory for saving processed data
save_path = Path('S:/clickbait-visual/bonsai_align')

Found 54 ttl.npy files
0 S:\clickbait-visual\preprocessed\7001\v1\ttls.npy
1 S:\clickbait-visual\preprocessed\7001\v2\ttls.npy
2 S:\clickbait-visual\preprocessed\7001\v3\ttls.npy
3 S:\clickbait-visual\preprocessed\7001\v4\ttls.npy
4 S:\clickbait-visual\preprocessed\7001\v5\ttls.npy
5 S:\clickbait-visual\preprocessed\7001\v6\ttls.npy
6 S:\clickbait-visual\preprocessed\7001\v7\ttls.npy
7 S:\clickbait-visual\preprocessed\7001\v8\ttls.npy
8 S:\clickbait-visual\preprocessed\7001\v10\ttls.npy
9 S:\clickbait-visual\preprocessed\7001\v11\ttls.npy
10 S:\clickbait-visual\preprocessed\7001\v12\ttls.npy
11 S:\clickbait-visual\preprocessed\7002\v2\ttls.npy
12 S:\clickbait-visual\preprocessed\7002\v3\ttls.npy
13 S:\clickbait-visual\preprocessed\7002\v4\ttls.npy
14 S:\clickbait-visual\preprocessed\7002\v5\ttls.npy
15 S:\clickbait-visual\preprocessed\7002\v6\ttls.npy
16 S:\clickbait-visual\preprocessed\7002\v7\ttls.npy
17 S:\clickbait-visual\preprocessed\7002\v8\ttls.npy
18 S:\clickbait-visual\preproc

In [11]:
# Filter paths to match ttl paths
def filter_paths(paths_to_filter, paths_to_reference):
    # Take paths if their final two parent directories match the reference paths
    filtered_paths = []
    for path in paths_to_filter:
        for ref_path in paths_to_reference:
            if (path.parents[1].name == ref_path.parents[1].name and 
                path.parents[0].name == ref_path.parents[0].name):
                filtered_paths.append(path)
                break
    return filtered_paths

# Filter paths
video_paths = filter_paths(video_paths, ttl_paths)
timestamps = filter_paths(timestamps, ttl_paths)
event_paths_a = filter_paths(event_paths_a, ttl_paths)
event_paths_b = filter_paths(event_paths_b, ttl_paths)

print(f"{len(video_paths)}")
print(f"{len(timestamps)}")
print(f"{len(event_paths_a)}")
print(f"{len(event_paths_b)}")

54
54
54
54


### Process events and assign TTL times to each observation

In [12]:
for session_idx in range(len(video_paths[:1])):
    #try:
    # Binarize ttl signal and crop to capture portion
    ttl_full = process_ttl(ttl_paths[session_idx], ttl_floor=10000, min_frame_duration=6, min_frame_spacing=6)
    ttl_crop, ttl_times = crop_ttl(ttl_full)

    event_data = process_events(session_idx, event_paths_a, event_paths_b)
    # Create a column of millisecond timestamps, derived from Bonsai timestamps and offset by the first ttl pulse time
    ttl_offset = len(ttl_times) - len(event_data)
    timestamp_ms =  ttl_times[ttl_offset:]
    event_data['timestamp_ms'] = timestamp_ms.astype(np.int32)

    # Set up save path
    mouse_id = ttl_paths[session_idx].parents[1].name
    session_id = ttl_paths[session_idx].parents[0].name
    event_path = save_path / mouse_id / session_id

    # if event path doesn't exist, create it
    if not event_path.exists():
        event_path.mkdir(parents=True, exist_ok=True)
    
    # Save event data
    event_data.to_csv(save_path / mouse_id / session_id / f'events.csv', index=False)
    print(f"Saved events for {mouse_id} {session_id} to {save_path / mouse_id / session_id / f'events.csv'}")
    #except:
     #   print(f"Error processing events for {video_paths[session_idx]}")
     #   continue

Saved events for 7001 v1 to S:\clickbait-visual\bonsai_align\7001\v1\events.csv
